In [33]:
# Importing the libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
%matplotlib inline

In [83]:
# Load dataset
dataset = pd.read_csv('AE_MBARAW_BS_LLY.csv')
dataset.head()

c:\users\nitin.b\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,invoice,sub_class
0,,BS-Non Trading_BS-Non Trading_NT - Consumables...
1,,BS-Non Trading_Gift Voucher_Gif Voucher_Gift V...
2,,Basics Apparels_Apparel Non Seasonal_Multipack...
3,,Basics Apparels_Apparel Non Seasonal_Multipack...
4,,Basics Apparels_Apparel Non Seasonal_Multipack...


In [84]:
# clean up invoice column
dataset['invoice'].replace('', np.nan, inplace=True)
print(dataset)

                 invoice                                          sub_class
0                    NaN  BS-Non Trading_BS-Non Trading_NT - Consumables...
1                    NaN  BS-Non Trading_Gift Voucher_Gif Voucher_Gift V...
2                    NaN  Basics Apparels_Apparel Non Seasonal_Multipack...
3                    NaN  Basics Apparels_Apparel Non Seasonal_Multipack...
4                    NaN  Basics Apparels_Apparel Non Seasonal_Multipack...
...                  ...                                                ...
6720991  LMSOS9842392215  Basics Apparels_Apparel Regular_Bodysuit_Bodysuit
6720992  LMSOS9842392215  Basics Non Apparels_Bath & Bedding_Blankets_Re...
6720993  LMSOS9842392215  Basics Non Apparels_Bath & Bedding_Towels & Ac...
6720994  LMSOS9842392820        Clothing Boys_New Toddler Boys_Tops_T-Shirt
6720995  LMSOS9842392820  Clothing Essentials_Essential Younger and Scho...

[6720996 rows x 2 columns]


In [85]:
# dropna's
dataset.dropna(subset=['invoice'], inplace=True)

In [86]:
dataset.head()

,invoice,sub_class
305,10003101000020181220,Basics Non Apparels_Feeding & Care_Feeding & S...
306,10003101000120180921,Basics Apparels_Apparel Non Seasonal_Multipack...
307,10003101000120180921,Basics Apparels_Apparel Regular_Accessories_Bo...
308,10003101000120180921,Basics Non Apparels_Feeding & Care_Baby Care_C...
309,10003101000120180921,Clothing Girls_Baby Girls_Bottoms_Pants


In [87]:
dataset.shape

(6720691, 2)

In [88]:
#Cleaning the data
dataset['sub_class'] = dataset['sub_class'].str.strip() 
dataset.dropna(axis=0, subset=['invoice'], inplace=True) 
dataset['invoice'] = dataset['invoice'].astype('str')
dataset.head()

,invoice,sub_class
305,10003101000020181220,Basics Non Apparels_Feeding & Care_Feeding & S...
306,10003101000120180921,Basics Apparels_Apparel Non Seasonal_Multipack...
307,10003101000120180921,Basics Apparels_Apparel Regular_Accessories_Bo...
308,10003101000120180921,Basics Non Apparels_Feeding & Care_Baby Care_C...
309,10003101000120180921,Clothing Girls_Baby Girls_Bottoms_Pants


In [9]:
# df=dataset.head(1000)
# df.shape

In [89]:
# collect sublcasses invoice wise in a list
df1 = dataset.groupby('invoice')['sub_class'].apply(list).reset_index(name='new')
df1.head()

,invoice,new
0,10003101000020181220,[Basics Non Apparels_Feeding & Care_Feeding & ...
1,10003101000120180921,[Basics Apparels_Apparel Non Seasonal_Multipac...
2,10003101000120181220,[Toys Boys & Girls_Boys Toys_Vehicle Toys_R/c ...
3,10003101000220180921,[Basics Apparels_Apparel Seasonal_Accessories_...
4,10003101000220181220,[Basics Non Apparels_Feeding & Care_Feeding & ...


In [90]:
df1.shape

(2240764, 2)

In [92]:
# use only the column of subclass collected in a list
data2 = df1['new']
data2.head()

0    [Basics Non Apparels_Feeding & Care_Feeding & ...
1    [Basics Apparels_Apparel Non Seasonal_Multipac...
2    [Toys Boys & Girls_Boys Toys_Vehicle Toys_R/c ...
3    [Basics Apparels_Apparel Seasonal_Accessories_...
4    [Basics Non Apparels_Feeding & Care_Feeding & ...
Name: new, dtype: object

In [93]:
boolean = df1["invoice"].is_unique
boolean

True

In [94]:
# create rules based on what support, confidence and lift you would like to see
rules = apriori(data2,min_support=0.001,min_confidence=0.1,min_lift=3,min_length=2)

In [95]:
results = list(rules)

In [96]:
len(results)

3082

In [97]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
ordered_results = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [98]:
ordered_results

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,Basics Apparels_Apparel Non Seasonal_Multipack...,Basics Apparels_Apparel Non Seasonal_Multipack...,0.002659,0.156203,4.499078
1,Basics Apparels_Apparel Non Seasonal_Multipack...,Basics Apparels_Apparel Non Seasonal_Multipack...,0.008334,0.240035,9.119237
2,Basics Apparels_Apparel Non Seasonal_Sleepwear...,Basics Apparels_Apparel Non Seasonal_Multipack...,0.003359,0.169325,4.877026
3,Basics Apparels_Apparel Non Seasonal_Multipack...,Basics Apparels_Apparel Regular_Accessories_Bi...,0.007640,0.220047,3.686117
4,Basics Apparels_Apparel Non Seasonal_Multipack...,Basics Apparels_Apparel Regular_Accessories_Bo...,0.007700,0.221795,3.244742
...,...,...,...,...,...
3077,Basics Non Apparels_Bath & Bedding_Blankets_Re...,Basics Non Apparels_Feeding & Care_Feeding & S...,0.001123,0.784289,10.037509
3078,Basics Apparels_Apparel Seasonal_Sleepsuit & R...,Basics Non Apparels_Feeding & Care_Feeding & S...,0.001125,0.784381,10.038680
3079,Basics Non Apparels_Bath & Bedding_Blankets_Re...,Basics Non Apparels_Feeding & Care_Feeding & S...,0.001063,0.799597,10.233423
3080,Basics Non Apparels_Bath & Bedding_Blankets_Re...,Basics Non Apparels_Feeding & Care_Feeding & S...,0.001046,0.795925,10.186429


In [99]:
# load final data into excel
ordered_results.to_excel("ae_mba_lly_v1.xlsx")